# Sound representation

The representation of a sound file is not as straightforward as that of an image - what is the correct representation for the entirety of the sonic experience that would be e.g. used for an analysis of "style" the way a GAN can turn a painting of a summer lawn into a wintery scene?  

If you wanted to "display" an audio file the way you do it with an image, by "looking" at it, you might want to play it:

In [ ]:
from IPython.core.display import display, HTML
def show_audio_with_controls(file_path):
    display(HTML("<audio controls><source src={} type='audio/mpeg'></audio>".format(file_path)))

In [ ]:
# The sound file is Groove Grove by Kevin MacLeod (incompetech.com)
# Licensed under Creative Commons: By Attribution 3.0 Licensehttps://creativecommons.org/licenses/by/3.0/
show_audio_with_controls('../example_snds/groove_grove_short.mp3')

While that's useful (you could play background music that way ;) ), it's clear that this is not a useful "representation" for machine learning. We need some other ways to translate the sound into digital information that the machine can evaluate.

We'll use a few libraries to dig into two main ways of representing sound for use with machine learning.

In [ ]:
import scipy.signal as signal
import librosa
import librosa.display

# We'll use some additional libraries for pretty plotting and other functionality
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load and display sound

The most familiar and common way of displaying sound in digital systems is as waveforms. A waveform is the digital representation of a sampled electric signal, the changing in voltage of the electric signal over time. Just like the resolution of an image with its colour channels and image size, an audio signal similarly has defining characteristics - most importantly, the sample rate (in Hz) and resolution (in bit).  
The sample rate is just that: the number of times per second the converter "looks" at the electric signal and measures it. The resolution gives the number of discrete values the measured signal voltage can take (see example).  
![Sampling of sine wave (single frequency tone)](../example_imgs/pcm.svg)  
(Image credit: Wikipedia commons)  
This image shows the sampling of a single frequency tone with a resolution of 4bit. The resolution limits the dynamic range of the signal, in the case of our 4-bit example, there are 7 different amplitudes that a sample can represent. That's like a volume switch with only 7 settings (plus off). This is the equivalent to the colour palette in an image.  
The sample rate determines which frequencies can still be resolved by individual samples - at high frequencies, the signal changes faster than the converter looks at it, so the amplitudes at that frequency cannot be properly captured. In principle, you need to sample at least at twice the highest frequency you want to capture to avoid getting badly sampled fragments in your audio signal (it's a bit more complicated than this, if you want to know more, look up the Nyquist-Shannon sampling theorem).  
"But wait," you might say, "that sounds like a waveform is just a timeseries. A bunch of numbers over time." and that is exactly right.
So let's load an audiofile. Librosa can import mp3 files and it converts them into a one-dimensional numpy array with floating point values between -1 and 1 (it does different things between -1 and 0 and 0 and 1 for stereo files than mono files, but we can neglect that for our purposes here).  

### Loading a sound file

In [ ]:
sound, samplerate = librosa.load('../example_snds/groove_grove_short.mp3')

Let's check out what that looks like:

In [ ]:
sound

In [ ]:
samplerate

In [ ]:
# Quick check: Let's find out how long the file is in seconds by dividing the length of the array by the samplerate
len(sound)/samplerate

Great! That checks out. Now let's have a look at the waveform.

### The waveform

In [ ]:
# Let's put the player up as well so we can "read along"
show_audio_with_controls('../example_snds/groove_grove_short.mp3')
plt.figure(figsize=(15, 6))
librosa.display.waveplot(sound, sr=samplerate)

That's pretty cool! You can actually see and hear the music together, you can see the accents, the long sounds and the short sounds and their different amplitudes. It's still not quite like reading music, though. If this was sheet music, you'd have melody notes along with chords and they would have different lengths. In the waveform, you can't see whether there's anything else going on when the big bass note is playing because it takes up all the amplitude. It would be cool if we could get a different representation that shows us the music in a way that's more like sheet music.  

### The Spectrogram

To get a better idea of what's happening, we can analyse the file for its frequency contents over time. Instead of looking at the amplitude of the entire waveform, i.e. the end result of all instruments (and thus frequencies) at once, we can do some clever mathematics to split the signal up into its component frequencies. To do that over time, we use a method called a "Short Time Fourier Transform". This function has a sliding window that moves across the file from start to finish and calculates its frequency content at each step.

In [ ]:
show_audio_with_controls('../example_snds/groove_grove_short.mp3')

plt.figure(figsize=(15, 5))
# Take the absolute values of the fourier transform found
D = np.abs(librosa.stft(sound))
librosa.display.specshow(D, y_axis='linear')
plt.colorbar(format='%+2.0f')
plt.title('Linear-frequency linear power spectrogram')

Hmmm... that doesn't seem very useful. There's just a few smudges of pink in the low frequency ranges and that's hardly visible.  
This might be good for a machine, but it isn't good for us. Music works differently in the brain! Pressure and sound are displayed in log scales and music works in octaves! Let's try that and see what happens.

In [ ]:
show_audio_with_controls('../example_snds/groove_grove_short.mp3')

plt.figure(figsize=(15, 5))
D = librosa.amplitude_to_db(np.abs(librosa.stft(sound)), ref=np.max)
librosa.display.specshow(D, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')

Boom! Much better. We can clearly see the percussive sounds that occur at the same time as the bass note and distinguish the double-note that occurs on the downbeat that was obscured in the waveform display. We can give this information to a neural network to learn a different feature representation than the waveform.

Librosa has a lot more features than just to display the waveforms and spectrograms, like a funciton to determine the tempo of a piece of music.  
If you want to find out more check out the documentation https://librosa.github.io/librosa/index.html.

In [ ]:
onset_env = librosa.onset.onset_strength(sound, sr=samplerate)
tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=samplerate)
tempo

# Exercise

Take the other example sound file (a montaged recording of a greeting in Esperanto from 1905) and see if you can follow the contents of the waveform and the spectrogram like with the music!

In [ ]:
show_audio_with_controls('example_snds/Mi_salutas_vin,_karaj_samideanoj,_fratoj_kaj_fratinoj_el_la_granda_tutmonda_homa_familio.ogg')

# What now?

Well, as we've said above - sound is, in principle, a fancy timeseries. Recurrent Neural Networks are the obvious choice for this type of problem, and they have been used quite successfully to produce models that can recognise human speech, for example.  
If you've worked with RNNs before, you'll remember that there are sequences of a specified length for inputs and outputs (in some systems, the length can even be variable). That works for audio signals as well, since they already come in little chunks - the samples. So to prepare your audio data for your RNN, you prepare a spectrogram of your signal and split the spectrogram into units of e.g. 20 milliseconds (depending on your architecture). You'll probably already guess that there's a rub. If you have a very high resolution audio signal (sample rates of up to 192kHz at 32 bit are used in some audio applications), that'll give you a very clear and well resolved signal to analyse. But your RNN will need to analyse all of those many chunks and it'll be slow or big or both. If you take a lower resolution, you'll get more data through, everything will be faster, but you might miss out on detail that could give you accuracy. There's no perfect answer to this (well, there is - as always, it's more data and more compute ;) ) but we've come pretty far with the compromises people have made so far. To start with, look up what the main frequency range of the signal (e.g. speech, different languages have different main ranges) is and you should be able to get the ballpark sample frequency if you double the top of that frequency range (no 192kHz required unless you want to translate batspeak).  
Alex Graves and his collaborators have been working on Speech-to-text machine learning for more than a decade, and their work is a good starting point:

https://arxiv.org/pdf/1303.5778.pdf (you'll recognise they're using spectrograms as well)

and here's a medium post by Adam Geitgey running through the whole idea in a lighter tone:

https://medium.com/@ageitgey/machine-learning-is-fun-part-6-how-to-do-speech-recognition-with-deep-learning-28293c162f7a